In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# DATA

In [2]:
app_train = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_train.csv')
app_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_test.csv')
prev_app = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\prev_app.csv')
pos_cash = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\pos_cash.csv')
bureau = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\bureau_balance_merge_bureau.csv')
ccb = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\credit_card_balance_final.csv')
inp = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\installments_payments_final.csv').drop(columns='Unnamed: 0')

In [3]:
inp = inp.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'})

In [4]:
full_app = pd.concat([app_train, app_test], axis = 0)
# full_app[(full_app['TARGET'] == 0) | (full_app['TARGET'] == 1)]

# Join All

In [5]:
full_train = pd.merge(full_app, bureau, on= 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,prev_app, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,inp, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,ccb, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,pos_cash, on = 'SK_ID_CURR', how = 'left')

# Fill nan

In [6]:
## check null
def check_missed_values(df):
    df_null_percentage = df.isnull().sum() / df.shape[0] * 100
    df_null_percentage = df_null_percentage.drop(df_null_percentage[df_null_percentage == 0].index).sort_values(ascending= False).reset_index()
    df_null_percentage.columns = ["Feature", "Percentage"]
    return df_null_percentage

null_df = check_missed_values(full_train)
null_df

,Feature,Percentage
0,AMT_PAYMENT_CURRENT_MIN,80.143800
1,AMT_PAYMENT_CURRENT_MAX,80.143800
2,AMT_PAYMENT_CURRENT_MEAN,80.143800
3,AMT_DRAWINGS_POS_CURRENT_MAX,80.117784
4,AMT_DRAWINGS_OTHER_CURRENT_MAX,80.117784
...,...,...
214,MONTHS_BALANCE_MEAN,5.883042
215,MONTHS_BALANCE_SUM,5.883042
216,COUNT_TIMES_OVER_DUE_MIN,5.883042
217,NAME_TYPE_SUITE,0.420148


In [7]:
drop_null_col =null_df[null_df["Percentage"] > 70]["Feature"].tolist()
full_train.drop(drop_null_col, axis=1, inplace= True)

# drop cot toan la gia tri 0

In [8]:
numeric_df = full_train._get_numeric_data()
numeric_df.drop([ "SK_ID_CURR", "TARGET"], axis= 1, inplace= True)

count_list =[]
for i in numeric_df.columns:
    fre_count = pd.Series(numeric_df[i].value_counts(normalize= True))
    count_list.append([i, fre_count.index[0], fre_count.values[0] * 100])

frequent_df = pd.DataFrame(count_list)
frequent_df.columns = ["Feature", "Value", "Frequent"]
frequent_df = frequent_df.sort_values(by= "Frequent",ascending= False)
to_drop = frequent_df[frequent_df["Frequent"] > 75]["Feature"].tolist()

full_train.drop(to_drop, axis=1, inplace= True)
full_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CNT_INSTALMENT_MAX,CNT_INSTALMENT_MIN,CNT_INSTALMENT_FUTURE_SUM,CNT_INSTALMENT_FUTURE_MEAN,CNT_INSTALMENT_FUTURE_MAX,PAYMENT_COUNTS_SUM,PAYMENT_COUNTS_MEAN,PAYMENT_COUNTS_MAX,PAYMENT_COUNTS_MIN,COMPLETED_STATUS_SUM
0,278621,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,12.0,6.0,1.0,0.333333,1.0,28.0,9.333333,12.0,8.0,2.0
1,139008,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,12.0,1.0,3.0,1.000000,3.0,21.0,7.000000,10.0,3.0,2.0
2,138348,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,24.0,10.0,14.0,2.800000,13.0,66.0,13.200000,18.0,10.0,3.0
3,64140,0.0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,10.0,6.0,0.0,0.000000,0.0,83.0,20.750000,55.0,8.0,4.0
4,219374,0.0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,10.0,10.0,0.0,0.000000,0.0,11.0,11.000000,11.0,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,150442,NaN,Cash loans,M,Y,Y,0,171000.0,521280.0,23089.5,...,24.0,2.0,3.0,0.272727,2.0,157.0,14.272727,24.0,3.0,9.0
307507,5217,NaN,Revolving loans,M,Y,Y,2,450000.0,900000.0,45000.0,...,24.0,4.0,1.0,0.250000,1.0,54.0,13.500000,25.0,5.0,3.0
307508,260741,NaN,Cash loans,F,N,Y,0,225000.0,202500.0,24030.0,...,8.0,1.0,1.0,0.125000,1.0,44.0,5.500000,9.0,2.0,7.0
307509,284794,NaN,Cash loans,M,N,Y,0,121500.0,254700.0,30357.0,...,15.0,4.0,0.0,0.000000,0.0,21.0,10.500000,16.0,5.0,2.0


In [9]:
numerical_data = full_train._get_numeric_data().drop(columns = ["SK_ID_CURR", "TARGET"])
cat_cols = [col for col in full_train.columns if col not in numerical_data.columns and col not in ["SK_ID_CURR", "TARGET"]]

In [10]:
# for column in numerical_data.columns[:-1]:
#     # Create a boxplot for the column
#     sns.boxplot(x=numerical_data[column])
#     # Set the title of the plot
#     plt.title(f'Boxplot for {column}')
#     # Show the plot
#     plt.show()

In [11]:
# Fill missing values for numerical columns with the mean
numerical_data.fillna(numerical_data.mean(), inplace=True)

# Fill missing values for categorical columns with the mode
for col in cat_cols:
    mode_value = full_train[col].mode()[0]  
    full_train[col].fillna(mode_value, inplace=True)


In [12]:
cat_cols

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FLAG_CONTACT',
 'INS_ON_TIME_GRADE_',
 'PAYMENT_GRADE_']

# Transformation

In [13]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
one_hot_encoder = OneHotEncoder
ordinal_encoder = OrdinalEncoder

X_ord = full_train[['INS_ON_TIME_GRADE_','PAYMENT_GRADE_']]
X_one_hot = full_train[[col for col in cat_cols if col not in ['INS_ON_TIME_GRADE_','PAYMENT_GRADE_']]]

y = full_train["TARGET"]

X_ord = OrdinalEncoder().fit_transform(X_ord)
X_one_hot = OneHotEncoder().fit_transform(X_one_hot)

In [14]:
X_ord = pd.DataFrame(X_ord)
X_one_hot = X_one_hot.toarray()
X_cat = pd.DataFrame(X_one_hot)
X_cat = pd.concat([X_ord,X_cat], axis= 1)

In [15]:
def check_values(column):
    # Check for infinity
    has_infinity = np.any(np.isinf(column))
    
    # Check for values too large for float64
    too_large_values = np.any(np.abs(column) > np.finfo(np.float64).max)
    
    return has_infinity, too_large_values

# Iterate over columns and check for infinity and large values
def find_col(data):
    cols = []
    for col in data.columns:
        has_infinity, too_large_values = check_values(numerical_data[col])
        
        if has_infinity:
            print(f"Column '{col}' contains infinity.")
            cols.append(col)

        
        if too_large_values:
            print(f"Column '{col}' contains values too large for dtype('float64').")
            if col not in cols:
                cols.append(col)
    return cols

col = find_col(numerical_data)

Column 'CAR_EMPLOYED_RATIO' contains infinity.
Column 'CAR_EMPLOYED_RATIO' contains values too large for dtype('float64').
Column 'PAYS_TO_INS_' contains infinity.
Column 'PAYS_TO_INS_' contains values too large for dtype('float64').


In [16]:
from sklearn.preprocessing import MinMaxScaler

X_num = numerical_data.drop(columns=col, axis=1 )

scaler = MinMaxScaler()
X_num = scaler.fit_transform(X_num)
X_num = pd.DataFrame(X_num)


In [17]:
full_data = pd.concat([X_num,X_cat],axis = 1)
full_data['SK_ID_CURR'] = full_train['SK_ID_CURR']
full_data['TARGET'] = full_train['TARGET']

# SPLIT

In [18]:
train = full_data[(full_data['TARGET'] == 0) | (full_data['TARGET'] == 1)]
test = full_data[~((full_data['TARGET'] == 0) | (full_data['TARGET'] == 1))]

In [19]:
train = train.set_index('SK_ID_CURR')
train.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\train.csv')

In [20]:
test = test.drop(columns='TARGET',axis = 1)
test = test.set_index('SK_ID_CURR')
test.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\test.csv')